In [19]:
import os
import pandas as pd
import nltk
nltk.download('wordnet')
import string
import contractions

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sarahbaskin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [21]:
albums = ["Taylor Swift", "Fearless (Taylor's Version)", "Speak Now (Taylor's Version)", 
          "Red (Taylor's Version)", "1989 (Taylor's Version)", "reputation", "Lover", 
          "Midnights", "folklore", "evermore", "The Tortured Poets Department"]

In [22]:
df = pd.read_csv('songs.csv')


In [23]:
in_album = df[df['Album'].isin(albums)]
df = in_album

In [24]:
lemmatizer = nltk.WordNetLemmatizer()

for lyric in df['Lyrics']:
    word_list=lyric.split()
    word_data = ''
    for word in word_list:
        # if not("]" in word or "[" in word):
        #     if word in contractions.contractions_dict:
        #         word = contractions.fix(word)
            word_data += word + " "
    # print(word_data.split())
    # nltk_tokens = nltk.word_tokenize(text=word_data, language='English', preserve_line=True)
    # nltk_tokens = [token.lower() for token in nltk_tokens if token.lower() not in stop_words]
    # nltk_tokens = [token for token in nltk_tokens if token not in string.punctuation]
    # nltk_tokens = [lemmatizer.lemmatize(token) for token in nltk_tokens]
    # nltk_tokens = [token for token in nltk_tokens if token != "''" and token != "``" and token != "'s"]

    lyric_column = df['Lyrics'] == lyric
    row_index = df.index[lyric_column]
    title = df[df['Lyrics'] == lyric]['Title'].values[0]
    album = df[df['Lyrics'] == lyric]['Album'].values[0]
    file_path = 'Songs/'+ album
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    song = os.path.join(file_path, title)
    with open(song, "w") as f:
        f.write('\n'.join(word_data.split()))

